In [2]:
%autoreload 2

In [3]:
from collections import defaultdict
import gzip
import os
import pickle
import sys
import time
import typing

import numpy as np
import tqdm.notebook as tqdm

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src.ast_utils import load_games_from_file, _extract_game_id

In [4]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
game_texts = list(load_games_from_file('../dsl/interactive-beta.pddl'))

In [6]:
GAME_START = '(define'
SETUP_SECTION = '(:setup'
PREFERENCES_SECTION = '(:constraints'
TERMINAL_SECTION = '(:terminal'
SCORING_SECTION = '(:scoring'
SECTION_KEYS = [SETUP_SECTION, PREFERENCES_SECTION, TERMINAL_SECTION, SCORING_SECTION]
OPEN_AND = '(and'
SECTION_SUFFIX = ')\n'
GAME_END_SUFFIX = '))\n'


DEFAULT_RANDOM_SEED = 33


def split_game(game_text: str, 
    start_section_str: str, end_section_str: typing.Union[None, str, typing.Sequence[str]],
    suffix: str = SECTION_SUFFIX, game_end_suffix: str = GAME_END_SUFFIX) -> typing.Tuple[str, str]:
    start_index = game_text.find(start_section_str)

    if end_section_str is None:
        end_index = None

    elif isinstance(end_section_str, str):
        end_index = game_text.find(end_section_str)

    else:
        end_indices = [game_text.find(s) for s in end_section_str]
        end_index = min([i for i in end_indices if i >= 0])

    if start_index == -1:
        start_index = end_index

    if end_index == -1:
        end_index = None

    if end_index is None:
        return game_text[:start_index] + start_section_str, game_end_suffix

    return game_text[:start_index] + start_section_str, suffix + game_text[end_index:]
        
        
def create_multi_game_prompt_suffix(game_texts: typing.List[str],
    n_games_before: int, n_games_after: int, 
    target_game_split_func: typing.Callable[..., typing.Tuple[str, str]],
    target_game_split_func_args: typing.Dict[str, typing.Any],
    game_filter_str: typing.Optional[str] = None,
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED
    ) -> typing.Tuple[str, str]:

    if rng is None:
        rng = np.random.default_rng(random_seed)

    if game_filter_str is not None:
        game_texts = [g for g in game_texts if game_filter_str in g]

    indices = rng.choice(len(game_texts), n_games_before + n_games_after + 1, replace=False)
    before_indices = indices[:n_games_before]
    target_index = indices[n_games_before]
    after_indices = indices[n_games_before + 1:]

    return create_multi_game_prompt_suffix_from_indices(game_texts, target_game_split_func, target_game_split_func_args, before_indices, target_index, after_indices)


def create_multi_game_prompt_suffix_from_indices(game_texts: typing.List[str], 
    target_game_split_func: typing.Callable[..., typing.Tuple[str, str]], 
    target_game_split_func_args: typing.Dict[str, typing.Any], 
    before_indices: typing.Sequence[int], 
    target_index: int, 
    after_indices: typing.Sequence[int]):

    before_games = [game_texts[i] for i in before_indices]
    target_prompt, target_suffix = target_game_split_func(game_texts[target_index], **target_game_split_func_args)
    after_games = [game_texts[i] for i in after_indices]

    before_games.append(target_prompt)
    after_games.insert(0, target_suffix)

    return '\n'.join(before_games), '\n'.join(after_games)


GAMES_WITH_SETUP = set([i for i, g in enumerate(game_texts) if SETUP_SECTION in g])
GAMES_WITH_SETUP_LIST = list(GAMES_WITH_SETUP)
GAMES_WITH_TERMINAL = set([i for i, g in enumerate(game_texts) if TERMINAL_SECTION in g])
GAMES_WITH_TERMINAL_LIST = list(GAMES_WITH_TERMINAL)


def create_all_prompts_for_game(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED) -> typing.Dict[str, typing.Any]:

    if rng is None:
        rng = np.random.default_rng(random_seed)

    setup_games = GAMES_WITH_SETUP_LIST[:]
    if game_index in setup_games:
        setup_games.remove(game_index)

    preferences_and_scoring_games = list(range(len(game_texts)))
    preferences_and_scoring_games.remove(game_index)

    terminal_games = GAMES_WITH_TERMINAL_LIST[:]
    if game_index in terminal_games:
        terminal_games.remove(game_index)

    setup_context_indices = rng.choice(setup_games, n_games_before + n_games_after, replace=False)
    preference_context_indices = rng.choice(preferences_and_scoring_games, n_games_before + n_games_after, replace=False)
    terminal_context_indices = rng.choice(terminal_games, n_games_before + n_games_after, replace=False)
    scoring_context_indices = rng.choice(preferences_and_scoring_games, n_games_before + n_games_after, replace=False)

    setup_prompt, setup_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
        {'start_section_str': SETUP_SECTION, 'end_section_str': PREFERENCES_SECTION}, 
        setup_context_indices[:n_games_before], game_index, setup_context_indices[n_games_before:])  # type: ignore

    preference_prompt, preference_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': PREFERENCES_SECTION, 'end_section_str': (TERMINAL_SECTION, SCORING_SECTION)},
            preference_context_indices[:n_games_before], game_index, preference_context_indices[n_games_before:])  # type: ignore

    terminal_prompt, terminal_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': TERMINAL_SECTION, 'end_section_str': SCORING_SECTION},
            terminal_context_indices[:n_games_before], game_index, terminal_context_indices[n_games_before:])  # type: ignore

    scoring_prompt, scoring_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': SCORING_SECTION, 'end_section_str': None},
            scoring_context_indices[:n_games_before], game_index, scoring_context_indices[n_games_before:])  # type: ignore

    return {
        SETUP_SECTION: dict(prompt=setup_prompt, suffix=setup_suffix, before_indices=setup_context_indices[:n_games_before], 
            target_index=game_index, after_indices=setup_context_indices[n_games_before:]),  
        PREFERENCES_SECTION: dict(prompt=preference_prompt, suffix=preference_suffix, before_indices=preference_context_indices[:n_games_before], 
            target_index=game_index, after_indices=preference_context_indices[n_games_before:]),
        TERMINAL_SECTION: dict(prompt=terminal_prompt, suffix=terminal_suffix, before_indices=terminal_context_indices[:n_games_before],
            target_index=game_index, after_indices=terminal_context_indices[n_games_before:]),
        SCORING_SECTION: dict(prompt=scoring_prompt, suffix=scoring_suffix, before_indices=scoring_context_indices[:n_games_before],
            target_index=game_index, after_indices=scoring_context_indices[n_games_before:])
    }



def create_iterative_prompts_for_game(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED) -> typing.Dict[str, typing.Any]:

    if rng is None:
        rng = np.random.default_rng(random_seed)

    preferences_and_scoring_games = list(range(len(game_texts)))
    preferences_and_scoring_games.remove(game_index)

    context_indices = rng.choice(preferences_and_scoring_games, n_games_before + n_games_after, replace=False)

    setup_prompt, setup_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
        {'start_section_str': SETUP_SECTION, 'end_section_str': PREFERENCES_SECTION}, 
        context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    preference_prompt, preference_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': PREFERENCES_SECTION, 'end_section_str': (TERMINAL_SECTION, SCORING_SECTION)},
            context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    terminal_prompt, terminal_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': TERMINAL_SECTION, 'end_section_str': SCORING_SECTION},
            context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    scoring_prompt, scoring_suffix = create_multi_game_prompt_suffix_from_indices(game_texts, split_game, 
            {'start_section_str': SCORING_SECTION, 'end_section_str': None},
            context_indices[:n_games_before], game_index, context_indices[n_games_before:])  # type: ignore

    return {
        SETUP_SECTION: dict(prompt=setup_prompt, suffix=setup_suffix, before_indices=context_indices[:n_games_before], 
            target_index=game_index, after_indices=context_indices[n_games_before:]),  
        PREFERENCES_SECTION: dict(prompt=preference_prompt, suffix=preference_suffix, before_indices=context_indices[:n_games_before], 
            target_index=game_index, after_indices=context_indices[n_games_before:]),
        TERMINAL_SECTION: dict(prompt=terminal_prompt, suffix=terminal_suffix, before_indices=context_indices[:n_games_before],
            target_index=game_index, after_indices=context_indices[n_games_before:]),
        SCORING_SECTION: dict(prompt=scoring_prompt, suffix=scoring_suffix, before_indices=context_indices[:n_games_before],
            target_index=game_index, after_indices=context_indices[n_games_before:])
    }



In [7]:
DEFAULT_CODEX_MODEL = "code-davinci-002"
DEFAULT_TEMPERATURE = 0.67
DEFAULT_MAX_TOKENS = 512
DEFAULT_STOP_SEQUENCES = [GAME_START, PREFERENCES_SECTION, TERMINAL_SECTION, SCORING_SECTION]
MAX_N = 10


DEFAULT_COMPLETION_KWARGS = dict(
    model=DEFAULT_CODEX_MODEL,
    temperature=DEFAULT_TEMPERATURE,
    max_tokens=DEFAULT_MAX_TOKENS,
    stop=DEFAULT_STOP_SEQUENCES,
)


def generate_codex_completions(prompt: str, suffix: str, n: int, 
    completion_kwargs: typing.Optional[typing.Dict[str, typing.Any]],
    ):

    if n > MAX_N:
        raise ValueError(f'n must be <= {MAX_N}')

    if completion_kwargs is None:
        completion_kwargs = DEFAULT_COMPLETION_KWARGS

    else:
        kwargs = DEFAULT_COMPLETION_KWARGS.copy()
        kwargs.update(completion_kwargs)
        completion_kwargs = kwargs

    completion_kwargs['prompt'] = prompt
    completion_kwargs['suffix'] = suffix
    completion_kwargs['n'] = n

    return openai.Completion.create(**completion_kwargs)
    

N_AUGMENTATIONS_PER_SECTION = {
    SETUP_SECTION: 3,
    PREFERENCES_SECTION: 6,
    TERMINAL_SECTION: 3,
    SCORING_SECTION: 3,
}

def generate_single_game_augmentations(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED,
    completion_kwargs_by_section: typing.Optional[typing.Dict[str, typing.Dict[str, typing.Any]]] = None,
    sleep_between_requests_seconds: float = 0.0,
    ):

    if completion_kwargs_by_section is None:
        completion_kwargs_by_section = defaultdict(dict)

    prompts_by_section = create_all_prompts_for_game(game_texts, game_index, n_games_before, n_games_after, rng, random_seed)
    
    for section in prompts_by_section:
        prompt = prompts_by_section[section]['prompt']
        suffix = prompts_by_section[section]['suffix']
        prompts_by_section[section]['results'] = generate_codex_completions(prompt, suffix, N_AUGMENTATIONS_PER_SECTION[section], completion_kwargs_by_section[section])
        time.sleep(sleep_between_requests_seconds)

    prompts_by_section['game_index'] = game_index
    prompts_by_section['game_id'] = _extract_game_id(game_texts[game_index])

    return prompts_by_section


def _iterative_generation_single_section(section_dict: typing.Dict[str, typing.Any],
    n_completions: int = 1, section_completions_kwarg: typing.Optional[typing.Dict[str, typing.Any]] = None):

    # TODO: handle `n_completions > 1`
    if n_completions > 1:
        raise NotImplementedError('n_completions > 1 not yet implemented')

    section_dict['results'] = generate_codex_completions(section_dict['prompt'], section_dict['suffix'], 
        n_completions, section_completions_kwarg)
    section_dict['completion'] = section_dict['results'].choices[0].text


def iterative_single_game_generation(game_texts: typing.List[str], 
    game_index: int, n_games_before: int, n_games_after: int, 
    rng: typing.Optional[np.random.Generator] = None, random_seed: int = DEFAULT_RANDOM_SEED,
    completion_kwargs_by_section: typing.Optional[typing.Dict[str, typing.Dict[str, typing.Any]]] = None,
    sleep_between_requests_seconds: float = 0.0,
    ):

    if completion_kwargs_by_section is None:
        completion_kwargs_by_section = defaultdict(dict)

    prompts_by_section = create_iterative_prompts_for_game(game_texts, game_index, n_games_before, n_games_after, rng, random_seed)

    sections = list(prompts_by_section.keys())
    sections.append(None)

    for current_section, next_section in zip(sections[:-1], sections[1:]):
        section_dict = prompts_by_section[current_section]
        _iterative_generation_single_section(section_dict, 1, completion_kwargs_by_section[current_section])

        if next_section is not None:
            next_updated_prompt = ' '.join((section_dict['prompt'], section_dict['completion']))
            while next_updated_prompt.count('(') < next_updated_prompt.count(')'):
                next_updated_prompt += SECTION_SUFFIX
            else:
                next_updated_prompt += '\n'
        
            next_updated_prompt += next_section
            prompts_by_section[next_section]['prompt'] = next_updated_prompt
            time.sleep(sleep_between_requests_seconds)

        else:
            final_generation_with_prompt = ' '.join((section_dict['prompt'], section_dict['completion']))
            final_generation = final_generation_with_prompt[final_generation_with_prompt.rfind(GAME_START):]
            while final_generation.count('(') < final_generation.count(')'):
                final_generation += ')'

            prompts_by_section['final_generation'] = final_generation
    
    prompts_by_section['game_index'] = game_index
    prompts_by_section['game_id'] = _extract_game_id(game_texts[game_index])

    return prompts_by_section
    

In [ ]:
random_seed = 23
output_path = f'/Users/guydavidson/projects/game-generation-modeling/dsl/codex/codex_augmentations_seed_{random_seed}.pkl.gz'
sleep_interval = 30
games_before = 3
games_after = 2
results_per_game = []


In [ ]:
done = False
while not done:
    try:
        last_index_found = results_per_game[-1]['game_index'] if len(results_per_game) > 0 else -1
        for index in tqdm.tnrange(last_index_found + 1, len(game_texts)):
            print(index)
            results_by_section = generate_single_game_augmentations(game_texts, index, games_before, games_after, 
                random_seed=random_seed, sleep_between_requests_seconds=sleep_interval)
            results_per_game.append(results_by_section)
            time.sleep(sleep_interval)
        done = True
    except openai.error.RateLimitError as e:
        print(f'Rate limit error: {e}, sleeping for {sleep_interval} seconds')
        time.sleep(sleep_interval)

original_output_path = output_path[:]
i = 0
while os.path.exists(output_path):
    output_path = original_output_path + f'_{i}'
    i += 1

with gzip.open(output_path, 'wb') as f:
    pickle.dump(results_per_game, f, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
random_seed = 77
output_path = f'/Users/guydavidson/projects/game-generation-modeling/dsl/codex/codex_single_game_iterative_seed_{random_seed}.pkl.gz'
sleep_interval = 30
games_before = 4
games_after = 1
iterative_results_per_game = []


In [9]:
sleep_interval = 30
done = False
while not done:
    try:
        last_index_found = iterative_results_per_game[-1]['game_index'] if len(iterative_results_per_game) > 0 else -1
        for index in tqdm.tnrange(last_index_found + 1, len(game_texts)):
            print(index)
            results_by_section = iterative_single_game_generation(game_texts, index, games_before, games_after, 
                random_seed=random_seed, sleep_between_requests_seconds=sleep_interval)
            iterative_results_per_game.append(results_by_section)
            time.sleep(sleep_interval)
        done = True
    except openai.error.RateLimitError as e:
        print(f'Rate limit error: {e}, sleeping for {sleep_interval} seconds')
        time.sleep(sleep_interval)
    except openai.error.Timeout as e:
        print(f'Timeout: {e}, sleeping for {sleep_interval} seconds')
        time.sleep(sleep_interval)
    

original_output_path = output_path[:]
i = 0
while os.path.exists(output_path):
    output_path = original_output_path + f'_{i}'
    i += 1

with gzip.open(output_path, 'wb') as f:
    pickle.dump(iterative_results_per_game, f, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/98 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [ ]:
with gzip.open(output_path, 'wb') as f:
    pickle.dump(iterative_results_per_game, f, protocol=pickle.HIGHEST_PROTOCOL)

# What's between these and a working augmented dataset:

1. Minor sanity checks, e.g. that the number of parentheses matches (or otherwise, remove trailing close parentheses)
2. Decide what to do about productions truncated for length (probably remove last partial preference)
3. Decide on a procedure to create games (sample from within the productions for a given game? between productions for differeng games?)
4. Generate a dataset with the procedure I have in mind, verify that the ASTs parse
5. Run it through the fitness function, compare the results to the real data
6. Create regrown corruptions for it, and repeat the previous synthetic experiments.


In [ ]:
import pickle
with open(f'/Users/guydavidson/Downloads/test.pkl', 'wb') as pickle_file:
    pickle.dump(results_by_section, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with gzip.open(output_path, 'rb') as pickle_file:
    test_results = pickle.load(pickle_file)

In [ ]:
test_results[10]['(:setup']['results']